<a href="https://colab.research.google.com/github/tanay0nSpark/ML_2022/blob/main/LinearRegression_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing some important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
# This library helps us in creating statiscal model of the dataset
# By this lib. we can check p-value and 100 - pvalue will give us
# contribution factor of the column or feature corresponding to label or ouput feature
import statsmodels.formula.api as snf